# 머신러닝 with Python 
## 09 앙상블 학습 배깅(Bootstrap aggregating) 연습- wine data set

##### 나이브 베이즈와 서포트 벡터 머신으로 해보았던 실험을 이어서 하기 위해 선형 회귀 모델을 데려와 해보기로 했다.

이어서 선형회귀 분류기를 사용해서 배깅을 연습해보기로 한다.

책 p.267~ (실습은 Gaussian Naive Bayse 모델로 진행되었지만 LinearRegression 으로 해보기로 한다.)


---

하단부에서는 gnb, svm과 마찬가지로 분류기의 개수에 따른 성능의 변화를 실험해 보았다.

배운점은 gnb, svm과 비교해보면서 적었다.

> 배운점1: 단독으로 정확도가 1.0이 나왔던 모형(LinearRegression)로만 배깅을 할경우 1.0보다 작은 정확도를 보이지 않았다.

> 배운점2: 배깅의 특성을 일반화 하기에 내가 실험한 데이터와 모형의 개수에는 한계가 크다. 배깅과 분류기, 분류기 개수들의 관계는 더 많은 데이터로 실험이 필요해보인다.

### 데이터 불러오기

In [1]:
from sklearn import datasets

In [2]:
raw_wine = datasets.load_wine() # 와인 데이터 가져오기

In [3]:
# 데이터 살펴보기
raw_wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
raw_wine.feature_names      # 데이터 셋 내 피처 이름들

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

### 피처, 타깃 데이터 지정

In [5]:
X = raw_wine.data
y = raw_wine.target

### 트레이닝/테스트 데이터 분할

In [6]:
from sklearn.model_selection import train_test_split                 # 분할을 위해 필요한 함수
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0)    # 분리. randomstate는 고정

### 데이터 표준화

In [7]:
from sklearn.preprocessing import StandardScaler         # 데이터 표준화를 위한 함수
std_scale = StandardScaler()                             # 표준화 스케일러 지정
std_scale.fit(X_tn)                                      # 트레이닝 피처를 기준으로 표준화를 적합 시도

X_tn_std = std_scale.transform(X_tn) 
X_te_std = std_scale.transform(X_te)                     # 트레인, 테스트 데이터 각각 적합시킨 표준화에 맞게 변형

### 데이터 학습

In [8]:
from sklearn.linear_model import LogisticRegression      # 선형 회귀 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류

In [9]:
# 배깅을 활용한 분류기 생성
clf_bagging_lr = BaggingClassifier(base_estimator = LogisticRegression(random_state=0), # base_estimator : 개별 학습기 입력. 선택하지 않으면 의사결정트리가 설정된다.
                                    n_estimators = 10,              # n_estimator : 개별 학습기의 개수. Default = 10
                                    random_state = 0)               # 고정

clf_bagging_lr.fit(X_tn_std, y_tn)                                 # 적합시키기

BaggingClassifier(base_estimator=LogisticRegression(random_state=0),
                  random_state=0)

### 데이터 예측

In [10]:
pred_bagging_lr = clf_bagging_lr.predict(X_te_std)                # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_bagging_lr)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [11]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_bagging_lr)                   # 실제값과 예측값을 넣음
print(accuracy)

1.0


### confusion matrix 확인

In [12]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_bagging_lr)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [13]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_bagging_lr)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



### 결과

1.0. 10개가 '참'으로 좋은 개수인건지 모르겠다.

## 실험1: 분류기 개수 2~10개 각각 성능 체크해보기

In [14]:
from sklearn.linear_model import LogisticRegression      # 선형 회귀 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류
from sklearn.metrics import accuracy_score               # 정확도 테스트 함수
from sklearn.metrics import classification_report        # 분류 리포트 확인을 위한 함수


for i in range(2, 11):
    # 배깅을 활용한 분류기 생성
    clf_bagging_lr = BaggingClassifier(base_estimator = LogisticRegression(random_state=0), 
                                        n_estimators = i,               # n_estimator : 개별 학습기의 개수
                                        random_state = 0)               # 고정
    
    # 데이터 학습
    clf_bagging_lr.fit(X_tn_std, y_tn)

    # 데이터 예측
    pred_bagging_lr = clf_bagging_lr.predict(X_te_std) 
    
    # 데이터 정확도 평가
    accuracy = accuracy_score(y_te, pred_bagging_lr)
    
    # 분류 리포트 확인
    class_report = classification_report(y_te, pred_bagging_lr)
    
    # 모델 상황 확인
    
    print("모델 분류기 개수: ", i )
    print("모델 정확도: ", accuracy)
    print("모델 분류 리포트")
    print(class_report)
    print("========================================================")

모델 분류기 개수:  2
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

모델 분류기 개수:  3
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

모델 분류기 개수:  4
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1      

 ### 결과
 
 개수가 10보다 작은 개수인데 변함이 없다. 이상으로 늘려본다.

## 실험2: 분류기가 11개 이상일 떄

In [15]:
from sklearn.linear_model import LogisticRegression      # 선형 회귀 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류
from sklearn.metrics import accuracy_score               # 정확도 테스트 함수
from sklearn.metrics import classification_report        # 분류 리포트 확인을 위한 함수


for i in range(11, 21):
    # 배깅을 활용한 분류기 생성
    clf_bagging_lr = BaggingClassifier(base_estimator = LogisticRegression(random_state=0), 
                                        n_estimators = i,               # n_estimator : 개별 학습기의 개수
                                        random_state = 0)               # 고정
    
    # 데이터 학습
    clf_bagging_lr.fit(X_tn_std, y_tn)

    # 데이터 예측
    pred_bagging_lr = clf_bagging_lr.predict(X_te_std) 
    
    # 데이터 정확도 평가
    accuracy = accuracy_score(y_te, pred_bagging_lr)
    
    # 분류 리포트 확인
    class_report = classification_report(y_te, pred_bagging_lr)
    
    # 모델 상황 확인
    
    print("모델 분류기 개수: ", i )
    print("모델 정확도: ", accuracy)
    print("모델 분류 리포트")
    print(class_report)
    print("========================================================")

모델 분류기 개수:  11
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

모델 분류기 개수:  12
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

모델 분류기 개수:  13
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1   

### 결과

SVM과 같은 특이한 결과를 보여주지 않았다. 처음부터 끝까지 정확도가 일관되게 나왔다고 한다. 

실제로 linear regreesion 단독으로 분류했던 wine데이터의 정확도도 1.0이었다. 선형회귀 모델에게 너무 쉬운 데이터인지도 모르겠다.

나중에 다른 복잡한 데이터로 다시 한 번 실험해봐야 하겠다...